# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [34]:
df = pd.read_csv('AviationData1.csv', low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50249 non-null  object 
 9   Airport.Name            52790 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87572 non-null  object 
 14  Make                    88826 non-null

In [35]:
# Inspect Missing Values (NaNs)
# Check if any NaN values exist in the entire DataFrame
print("\nAny NaN values in the DataFrame:", df.isnull().values.any())


Any NaN values in the DataFrame: True


In [36]:
#Count NaN values per column
print("\nNaN values per column:")
print(df.isnull().sum())


NaN values per column:
Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     52
Country                     226
Latitude                  54507
Longitude                 54516
Airport.Code              38640
Airport.Name              36099
Injury.Severity            1000
Aircraft.damage            3194
Aircraft.Category         56602
Registration.Number        1317
Make                         63
Model                        92
Amateur.Built               102
Number.of.Engines          6084
Engine.Type                7077
FAR.Description           56866
Schedule                  76307
Purpose.of.flight          6192
Air.carrier               72241
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Uninjured            5912
Weather.Condition          4492
Broad.phase.of.flight     27165
Report.Status              6381
Publication.Date

In [37]:
print(df.describe())

       Number.of.Engines  Total.Fatal.Injuries  Total.Serious.Injuries  \
count       82805.000000          77488.000000            76379.000000   
mean            1.146585              0.647855                0.279881   
std             0.446510              5.485960                1.544084   
min             0.000000              0.000000                0.000000   
25%             1.000000              0.000000                0.000000   
50%             1.000000              0.000000                0.000000   
75%             1.000000              0.000000                0.000000   
max             8.000000            349.000000              161.000000   

       Total.Minor.Injuries  Total.Uninjured  
count          76956.000000     82977.000000  
mean               0.357061         5.325440  
std                2.235625        27.913634  
min                0.000000         0.000000  
25%                0.000000         0.000000  
50%                0.000000         1.000000  
75% 

## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [38]:
# Inspect unique values in 'Aircraft_Category', Aircraft_Type, Make
print(df['Amateur.Built'].value_counts())
print( " --------- ")
print(df['Aircraft.Category'].value_counts())
print( " --------- ")
print(df['Make'].value_counts().head(20))

No     80312
Yes     8475
Name: Amateur.Built, dtype: int64
 --------- 
Airplane             27617
Helicopter            3440
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              30
Unknown                 14
WSFT                     9
Powered-Lift             5
Blimp                    4
UNK                      2
Rocket                   1
ULTR                     1
Name: Aircraft.Category, dtype: int64
 --------- 
Cessna               22227
Piper                12029
CESSNA                4922
Beech                 4330
PIPER                 2841
Bell                  2134
Boeing                1594
BOEING                1151
Grumman               1094
Mooney                1092
BEECH                 1042
Robinson               946
Bellanca               886
Hughes                 795
Schweizer              629
Air Tractor            595
BELL                   588
Mcdonnell Doug

In [39]:
print(df[['Aircraft.Category', 'Amateur.Built', 'Event.Date']].head())
print(df[['Aircraft.Category', 'Amateur.Built', 'Event.Date']].info())

  Aircraft.Category Amateur.Built Event.Date
0               NaN            No   10/24/48
1               NaN            No    7/19/62
2               NaN            No    8/30/74
3               NaN            No    6/19/77
4               NaN            No     8/2/79
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Aircraft.Category  32287 non-null  object
 1   Amateur.Built      88787 non-null  object
 2   Event.Date         88889 non-null  object
dtypes: object(3)
memory usage: 2.0+ MB
None


In [40]:
df['Aircraft.Category'] = df['Aircraft.Category'].fillna("Unknown")

In [41]:
df['Amateur.Built'] = df['Amateur.Built'].fillna("No")

In [42]:
from datetime import datetime
df['Event.Date'] = pd.to_datetime(df['Event.Date'], errors='coerce')
df = df.dropna(subset=['Event.Date'])

In [43]:
#Apply filters
df_filtered = df[df['Aircraft.Category'].str.contains("Airplane", case=False, na=False)]

# Client wants professional builds only
df_filtered = df_filtered[df_filtered['Amateur.Built'].str.lower() == "no"]

# Client wants events in the last 40 years
cutoff_date = datetime.now() - pd.DateOffset(years=40)
df_filtered = df_filtered[df_filtered['Event.Date'] >= cutoff_date]

# Final filtered dataset shape
print("Filtered dataset shape:", df_filtered.shape)

# Preview final result
print(df_filtered.head())

Filtered dataset shape: (21441, 31)
             Event.Id Investigation.Type Accident.Number Event.Date  \
13114  20001214X37718           Accident      ATL85MA286 1985-09-29   
14259  20001213X33054           Accident      FTW86FA050 1986-03-29   
14357  20001213X33306           Accident      LAX86LA166 1986-04-08   
14420  20001213X33276           Accident     FTW86FA066B 1986-04-15   
14421  20001213X33276           Accident     FTW86FA066A 1986-04-15   

              Location        Country Latitude Longitude Airport.Code  \
13114  JENKINSBURG, GA  United States      NaN       NaN          NaN   
14259   SEAGOVILLE, TX  United States      NaN       NaN          59F   
14357         MESA, AZ  United States      NaN       NaN          FFZ   
14420     HANKAMER, TX  United States      NaN       NaN          NaN   
14421     HANKAMER, TX  United States      NaN       NaN          NaN   

                    Airport.Name  ...   Purpose.of.flight Air.carrier  \
13114  WEST WIND SPORT PA

### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [44]:
injury_cols = ["Total.Fatal.Injuries", "Total.Serious.Injuries", 
               "Total.Minor.Injuries", "Total.Uninjured"]
for col in injury_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)

In [45]:
df["Total_Passengers"] = ( df["Total.Fatal.Injuries"] + df["Total.Serious.Injuries"] +df["Total.Minor.Injuries"] +df["Total.Uninjured"] )
df["Total_Passengers"].info()
df["Total_Passengers"].isnull()
#If Total_Passengers > 0  valid flight data.
#If Total_Passengers = 0  treat it as NaN (missing), not a real 0 
#because (Fatal + Serious) / Total_Passengers means we’d hit a divideby zero error or artificially inflate the injury ratio.
df.loc[df["Total_Passengers"] == 0, "Total_Passengers"] = np.nan

<class 'pandas.core.series.Series'>
RangeIndex: 88889 entries, 0 to 88888
Series name: Total_Passengers
Non-Null Count  Dtype
--------------  -----
88889 non-null  int64
dtypes: int64(1)
memory usage: 694.6 KB


In [46]:
df["Fatal_Serious_Fraction"] = ((df["Total.Fatal.Injuries"] + df["Total.Serious.Injuries"]) / df["Total_Passengers"])
# Fill NaN (from missing passengers) with 0
df["Fatal_Serious_Fraction"] = df["Fatal_Serious_Fraction"].fillna(0)


This metric ranges from 0 to 1, 
where:0 → no fatal/serious injuries,
      1 → all passengers fatally/seriously injured,
      0.5 → half of passengers fatally/seriously injured.

**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [52]:
print(df["Aircraft.damage"].head(10))
df["Aircraft.damage"] = df["Aircraft.damage"].str.strip().str.lower()

0      destroyed
1      destroyed
2      destroyed
3      destroyed
4      destroyed
5    substantial
6      destroyed
7    substantial
8    substantial
9    substantial
Name: Aircraft.damage, dtype: object


"Aircraft.Damage" is a text column (categorical values like "destroyed", "substantial", "minor", or even NaN).
So we convert the category into a binary numeric flag:
If the aircraft was "destroyed" → 1
If anything else (substantial, minor, unknown, missing) → 0

In [55]:
# using apply function and anonymous function
df["Destroyed"] = df["Aircraft.damage"].apply(lambda x: 1 if x == "destroyed" else 0)


### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [56]:
# Step 1: Standardize casing and remove whitespace
df["Make"] = df["Make"].str.upper().str.strip()

# Step 2: Replace unknowns with NaN
df["Make"].replace({"UNK": np.nan, "UNKNOWN": np.nan}, inplace=True)

# Step 3: Count occurrences
make_counts = df["Make"].value_counts()

# Step 4: Keep only makes with >= 50 records, else label as "OTHER"
df["Make_Cleaned"] = df["Make"].apply(lambda x: x if pd.notna(x) and make_counts.get(x, 0) >= 50 else "OTHER")

# Quick check
print(df["Make_Cleaned"].value_counts().head(20))

CESSNA               27149
PIPER                14870
OTHER                14239
BEECH                 5372
BOEING                2745
BELL                  2722
MOONEY                1334
ROBINSON              1230
GRUMMAN               1172
BELLANCA              1045
HUGHES                 932
SCHWEIZER              773
AIR TRACTOR            691
AERONCA                636
MCDONNELL DOUGLAS      608
MAULE                  589
CHAMPION               519
STINSON                439
AERO COMMANDER         429
DE HAVILLAND           422
Name: Make_Cleaned, dtype: int64


### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [62]:
df = df.dropna(subset=["Model"])
# Count unique models
print(df["Model"].nunique())

# Get top 20 most common models
print(df["Model"].value_counts().head(20))

12315
152          2367
172          1756
172N         1164
PA-28-140     932
150           829
172M          798
172P          689
182           659
180           622
150M          585
PA-18         581
PA-18-150     578
PA-28-180     572
PA-28-161     569
PA-28-181     532
206B          524
737           489
PA-38-112     469
150L          461
G-164A        460
Name: Model, dtype: int64


In [63]:
model_make_check = df.groupby("Model")["Make"].nunique().sort_values(ascending=False)
print(model_make_check.head(20))

Model
RV-4                 92
KR-2                 87
RV-6A                78
RV-6                 74
CHALLENGER II        61
AVID FLYER           48
VARIEZE              47
RV-8                 39
THORP T-18           32
KITFOX               31
SKYBOLT              30
LONG EZ              29
VARI-EZE             29
DRAGONFLY            28
QUICKIE              28
LONG-EZ              28
SONEX                28
SEAREY               28
CHRISTEN EAGLE II    26
STEEN SKYBOLT        25
Name: Make, dtype: int64


In [65]:
df["Make_Model"] = df["Make"].astype(str) + " " + df["Model"].astype(str)

### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [66]:
for col in [
    "Engine.Type", 
    "Weather.Condition", 
    "Number.of.Engines", 
    "Purpose.of.flight", 
    "Broad.phase.of.flight"
]:
    print(df[col].value_counts(dropna=False).head(15))


Reciprocating      69506
NaN                 7015
Turbo Shaft         3609
Turbo Prop          3390
Turbo Fan           2478
Unknown             2049
Turbo Jet            703
None                  19
Geared Turbofan       12
Electric              10
LR                     2
NONE                   2
Hybrid Rocket          1
UNK                    1
Name: Engine.Type, dtype: int64
VMC    77264
IMC     5974
NaN     4442
UNK      855
Unk      262
Name: Weather.Condition, dtype: int64
1.0    69550
2.0    11072
NaN     6031
0.0     1226
3.0      483
4.0      431
8.0        3
6.0        1
Name: Number.of.Engines, dtype: int64
Personal               49423
Instructional          10599
Unknown                 6793
NaN                     6141
Aerial Application      4710
Business                4016
Positioning             1645
Other Work Use          1264
Ferry                    812
Aerial Observation       794
Public Aircraft          720
Executive/corporate      553
Flight Test              

In [70]:
df["Engine.Type"] = (df["Engine.Type"].str.strip().str.upper().replace({"UNK": None, "UNKNOWN": None}))

# Weather.Condition
df["Weather.Condition"] = (df["Weather.Condition"].str.strip().str.upper().replace({"UNK": None, "UNKNOWN": None}))

# Number.of.Engines
# Convert to numeric if it has text
df["Number.of.Engines"] = pd.to_numeric( df["Number.of.Engines"], errors="coerce")

# Purpose.of.flight
df["Purpose.of.flight"] = (df["Purpose.of.flight"].str.strip() .str.title()) # Standardize capitalization

# Broad.phase.of.flight
df["Broad.phase.of.flight"] = (df["Broad.phase.of.flight"].str.strip().str.title().replace({"Taxiing": "Taxi"}))

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [71]:
na_percent = df.isna().mean().sort_values(ascending=False) * 100
print(na_percent)

# Quick look at columns with >50% missing
print(na_percent[na_percent > 50])

Schedule                  85.884658
Air.carrier               81.293287
FAR.Description           64.021307
Longitude                 61.321892
Latitude                  61.311756
Airport.Code              43.444035
Airport.Name              40.590335
Broad.phase.of.flight     30.522428
Publication.Date          15.502776
Engine.Type               10.208678
Report.Status              7.137629
Purpose.of.flight          6.915774
Number.of.Engines          6.791896
Weather.Condition          6.260347
Aircraft.damage            3.574445
Total_Passengers           1.442616
Registration.Number        1.440364
Injury.Severity            1.103641
Country                    0.253387
Location                   0.058561
Make                       0.050677
Total.Uninjured            0.000000
Destroyed                  0.000000
Make_Cleaned               0.000000
Fatal_Serious_Fraction     0.000000
Event.Id                   0.000000
Total.Minor.Injuries       0.000000
Total.Serious.Injuries     0

In [74]:
df_cleaned = df.dropna(thresh=len(df)*0.5, axis=1)

print(df_cleaned.columns)

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Airport.Code', 'Airport.Name',
       'Injury.Severity', 'Aircraft.damage', 'Aircraft.Category',
       'Registration.Number', 'Make', 'Model', 'Amateur.Built',
       'Number.of.Engines', 'Engine.Type', 'Purpose.of.flight',
       'Total.Fatal.Injuries', 'Total.Serious.Injuries',
       'Total.Minor.Injuries', 'Total.Uninjured', 'Weather.Condition',
       'Broad.phase.of.flight', 'Report.Status', 'Publication.Date',
       'Total_Passengers', 'Fatal_Serious_Fraction', 'Destroyed',
       'Make_Cleaned', 'Make_Model'],
      dtype='object')


### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized